In [1]:
import pandas as pd

In [23]:
from neo4j import GraphDatabase


class Neo4j:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()


neo4j = Neo4j("bolt://localhost:7687", "neo4j", "password")


def query(query):
    return neo4j.driver.execute_query(query)

In [4]:
df = pd.read_csv("DATA/SPINchoice.csv")

In [12]:
df.head(1)

,TimeStamp,FirstName,LastName,Comment,level,Choice01,Choice02,Choice03,Choice04,Choice05,IntensiveChoice01,IntensiveChoice02,IntensiveChoice03
0,2023/10/19 8:53:56,Aaro,Luhtanen,NaN,NaN,N5 漢字 Kanji for JLPT N5（A1.1-A1.2）,JLPT文法 (JLPT Grammar) (A1.3-B2),N4 L11-20 漢字 Kanji for JLPT N4 L11-20 (A2.1-A...,生活の中の読解 Reading in Everyday Life (A1-B1),ことば（Vocabulary ）,*JLPT Reading(A1.3-B2),*Communication(A1-B2),*Learning Japanese w/ Games and Board Games(A1...


In [15]:
summary = df.groupby("Choice01").value_counts()

In [16]:
summary = pd.crosstab(df["FirstName"], df["Choice01"])

In [21]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})

In [22]:
df.to_json()

'{"A":{"0":1,"1":2,"2":3},"B":{"0":4,"1":5,"2":6}}'

In [147]:
result = query(
    "MATCH (s:Student)-[w:WANTS]->(c:Course) WITH s, COLLECT({choice: w.choice, course: c.name}) as choices RETURN s as student, choices ORDER BY s.firstName, s.lastName"
)
studentChoices = [record.data() for record in result.records]

In [148]:
studentChoices[0]

{'student': {'firstName': 'Aaro',
  'lastName': 'Luhtanen',
  'level': 'A1-1',
  'active': True,
  'id': 'a5f07cc9-a9ef-434b-aac0-92e261cb67c6'},
 'choices': [{'course': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）',
   'choice': 'Choice01'},
  {'course': 'JLPT文法 (JLPT Grammar) (A1.3-B2)', 'choice': 'Choice02'},
  {'course': 'ことば（Vocabulary ）', 'choice': 'Choice05'},
  {'course': '生活の中の読解 Reading in Everyday Life (A1-B1)', 'choice': 'Choice04'},
  {'course': 'N4 L11-20 漢字 Kanji for JLPT N4  L11-20 (A2.1-A2.3)',
   'choice': 'Choice03'},
  {'course': '*JLPT Reading(A1.3-B2)', 'choice': 'IntensiveChoice01'},
  {'course': '*Communication(A1-B2)', 'choice': 'IntensiveChoice02'},
  {'course': '*Learning Japanese w/ Games and Board Games(A1-B2)',
   'choice': 'IntensiveChoice03'}]}

In [59]:
d = {item["choice"]: item["course"] for item in choices}

In [90]:
flatChoices = []
for item in studentChoices:
    choices = {item["choice"]: item["course"] for item in item["choices"]}
    flatChoices += [{**item["student"], **choices}]

In [145]:
result = query(
    " MATCH (s:Student) RETURN s, COLLECT {MATCH(s)-[r:WANTS]->(c:Course) RETURN r ORDER BY r.choice} as choices"
)

choices = [record.data() for record in result.records]
df = pd.DataFrame(choices)

In [146]:
result[0]

[<Record s=<Node element_id='4:cfc647e9-47d5-478a-941e-2695e1c588f6:65' labels=frozenset({'Student'}) properties={'firstName': 'Aaro', 'lastName': 'Luhtanen', 'level': 'A1-1', 'active': True, 'id': 'a5f07cc9-a9ef-434b-aac0-92e261cb67c6'}> choices=[<Relationship element_id='5:cfc647e9-47d5-478a-941e-2695e1c588f6:5077' nodes=(<Node element_id='4:cfc647e9-47d5-478a-941e-2695e1c588f6:65' labels=frozenset({'Student'}) properties={'firstName': 'Aaro', 'lastName': 'Luhtanen', 'level': 'A1-1', 'active': True, 'id': 'a5f07cc9-a9ef-434b-aac0-92e261cb67c6'}>, <Node element_id='4:cfc647e9-47d5-478a-941e-2695e1c588f6:787' labels=frozenset() properties={}>) type='WANTS' properties={'ranking': 10, 'choice': 'Choice01'}>, <Relationship element_id='5:cfc647e9-47d5-478a-941e-2695e1c588f6:5267' nodes=(<Node element_id='4:cfc647e9-47d5-478a-941e-2695e1c588f6:65' labels=frozenset({'Student'}) properties={'firstName': 'Aaro', 'lastName': 'Luhtanen', 'level': 'A1-1', 'active': True, 'id': 'a5f07cc9-a9ef-434b